In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import pandas as pd
import subprocess, os, glob

from scipy.fftpack import fftn, fftfreq, fftshift, ifftn
from scipy.stats import binned_statistic

# initial thermal spectrum

def pkphi(k, T_initial=4.0, lambdaPRS=1.0, scalefactor=1.0):
    T = T_initial / scalefactor
    m_eff_squared = lambdaPRS * (T**2 / 3 - 1)
    omega = np.sqrt(k**2 + scalefactor**2*m_eff_squared);
    bose = 1 / (np.exp(omega / T) - 1);
    return bose / omega

def pkphidot(k, T_initial=4.0, lambdaPRS=1.0, scalefactor=1.0):
    T = T_initial / scalefactor
    m_eff_squared = lambdaPRS * (T**2 / 3 - 1)
    omega = np.sqrt(k**2 + scalefactor**2*m_eff_squared);
    bose = 1 / (np.exp(omega / T) - 1);
    return bose * omega

# helper functions for plotting:

def plot_settings(figsize=(7,5)):
    ''' For nicer plots.
    '''
    font = {'size'   : 20, 'family':'STIXGeneral'}
    axislabelfontsize='large'
    matplotlib.rc('font', **font)
    matplotlib.mathtext.rcParams['legend.fontsize']='small'
    matplotlib.rcParams['mathtext.fontset'] = 'cm'
    plt.rcParams['figure.figsize'] = figsize

def axion_plot_settings(tau):
    plt.xlabel(r"Comoving distance $x/f_a$")
    plt.title(fr" $\tau/\tau_0 = {round(tau, 2)}$", loc = 'left')
    cbar = plt.colorbar(fraction=0.05, pad=0.04, shrink=0.75)
    cbar.set_ticks([np.pi-0.001, np.pi/2, 0, -np.pi/2, -np.pi+0.001])
    cbar.set_ticklabels([r"$\pi$", r"$\pi/2$","0",r"$-\pi/2$", r"$-\pi$"])
    cbar.set_label(r"$a(x)/f_a$", rotation=0, labelpad=15)

def scalar_field_plot_settings(tau):
    plt.xlabel(r"Comoving distance $x/f_a$")
    plt.title(fr" $\tau/\tau_0 = {round(tau, 2)}$", loc = 'left')
    plt.colorbar(fraction=0.05, pad=0.04, shrink=0.75)

def power_spectrum_plot_settings(tau=None):
    plt.xlabel(r"$k$")
    plt.ylabel(r"$P\,(k)$", rotation=0, labelpad=30)
    if tau is not None:
        plt.title(fr" $\tau/\tau_0 = {round(tau, 2)}$", loc = 'left')

def xi_plot_settings():
    plt.xlabel(r"$\tau/\tau_0$")
    plt.ylabel(r"$\xi$", rotation=0, labelpad=20)

# helper functions to read in raw bytes:

def convert_format2D(arr, N):
    ret = np.zeros((N, N))
    for i in range(N):
        for j in range(N):
            ret[i,j] = arr[i + N * j]
    
    return ret

def read_snapshots2D(N, func=lambda p1, p2: np.arctan2(p1, p2)):
    ''' Note: Appends axion field to snapshots by default.
    '''
    snapshots = []
    count = 0
    done = False
    while not done:
        try:
            phi1 = convert_format2D(np.fromfile(f"snapshot{count}-phi1", dtype=np.float64), N)
            phi2 = convert_format2D(np.fromfile(f"snapshot{count}-phi2", dtype=np.float64), N)
            snapshots.append(func(phi1, phi2))
            count += 1
        except:
            done = True
    
    return snapshots

def convert_format3D(arr, N):
    ret = np.zeros((N, N, N))
    for i in range(N):
        for j in range(N):
            for k in range(N):
                ret[i,j,k] = arr[(i * N + j) * N + k]

    return ret

def read_snapshots3D(N, func=lambda p1, p2: np.arctan2(p1, p2)):
    ''' Note: Appends axion field to snapshots by default.
    '''
    snapshots = []
    count = 0
    done = False
    while not done:
        try:
            phi1 = convert_format3D(np.fromfile(f"snapshot{count}-phi1", dtype=np.float64), N)
            phi2 = convert_format3D(np.fromfile(f"snapshot{count}-phi2", dtype=np.float64), N)
            snapshots.append(func(phi1, phi2))
            count += 1
        except:
            done = True
    
    return snapshots

# make a movie out of saved images:

def run_ffmpeg(filename, framerate=5):

    subprocess.call(['ffmpeg', '-framerate', str(framerate), '-i', 'tmp_img_%02d.png', '-pix_fmt', 'yuv420p', filename])

    # cleanup image files:
    for fname in glob.glob("tmp_img*"):
        os.remove(fname)


In [ ]:
# Show 2D snapshots:
%matplotlib inline
plot_settings(figsize=(13,12))

make_movie = False

# snapshots = read_snapshots2D(128, func=lambda p1, p2: p1)
snapshots = read_snapshots2D(128)
timings = pd.read_csv("snapshot-timings.csv")
tau = np.array(timings['tau'].tolist())

for i in range(len(snapshots)):
#     plt.imshow(snapshots[i], cmap=matplotlib.cm.viridis, interpolation='none')
#     scalar_field_plot_settings(tau[i])
    plt.imshow(snapshots[i], cmap=matplotlib.cm.twilight, interpolation='none', vmin=-np.pi, vmax=np.pi)
    axion_plot_settings(tau[i])
    
    if make_movie:
        plt.savefig("tmp_img_%02d.png" % count_id, facecolor='white')

    plt.show()


if make_movie:
    framerate = 5
    run_ffmpeg("video.mp4", framerate=framerate)


In [ ]:
# Show 3D snapshots as 2D slices:
%matplotlib inline
plot_settings(figsize=(13,12))

make_movie = False

# snapshots = read_snapshots3D(128, func=lambda p1, p2: p1)
snapshots = read_snapshots3D(128)
timings = pd.read_csv("snapshot-timings.csv")
tau = np.array(timings['tau'].tolist())

for i in range(len(snapshots)):
#     plt.imshow(snapshots[i][0], cmap=matplotlib.cm.viridis, interpolation='none')
#     scalar_field_plot_settings(tau[i])
    plt.imshow(snapshots[i][0], cmap=matplotlib.cm.twilight, interpolation='none', vmin=-np.pi, vmax=np.pi)
    axion_plot_settings(tau[i])

    if make_movie:
        plt.savefig("tmp_img_%02d.png" % count_id, facecolor='white')

    plt.show()

if make_movie:
    framerate = 5
    run_ffmpeg("video.mp4", framerate=framerate)

In [ ]:
# Read single snapshot:
%matplotlib inline
plot_settings(figsize=(13,12))

p1 = convert_format2D(np.fromfile("snapshot-final-phi1", dtype=np.float64), 128)
p2 = convert_format2D(np.fromfile("snapshot-final-phi2", dtype=np.float64), 128)

ax = np.arctan2(p1,p2)
sx = np.sqrt(p1**2 + p2**2)

plt.imshow(ax, cmap=matplotlib.cm.twilight, interpolation='none', vmin=-np.pi, vmax=np.pi)
axion_plot_settings()
plt.show()

In [ ]:
# Time-series observables:
%matplotlib inline
plot_settings()

ret = pd.read_csv("time-series.csv")
t = np.array(ret['time'].tolist())
tau = a = np.array(ret['scalefactor'].tolist())
xi = np.array(ret['xi'].tolist())

plt.plot(tau, xi, "-")
xi_plot_settings()
plt.show()

if False:
    # need to divide by a when fields are rescaled:
    phi1_bar = np.array(ret['phi1_bar'].tolist()) / a
    phi2_bar = np.array(ret['phi2_bar'].tolist()) / a
    phidot1_bar = np.array(ret['phidot1_bar'].tolist())
    phidot2_bar = np.array(ret['phidot2_bar'].tolist())
    axion_bar = np.array(ret['axion_bar'].tolist())
    saxion_bar = np.array(ret['saxion_bar'].tolist()) / a

    plt.scatter(phi1_bar, phi2_bar, marker=".", c=t, cmap=matplotlib.cm.viridis)
    plt.colorbar(fraction=0.05, pad=0.04, shrink=0.75)
    plt.xlabel(r"$\langle\phi_1\rangle$")
    plt.ylabel(r"$\langle\phi_2\rangle$", rotation=0, labelpad=30)
    plt.show()

    plt.plot(t, phi1_bar, ".-")
    plt.xlabel(r"$t/f_a$")
    plt.ylabel(r"$\langle \phi_1(x)\rangle$", rotation=0, labelpad=30)
    plt.show()

    plt.plot(t, saxion_bar, ".-")
    plt.xlabel(r"$t/f_a$")
    plt.ylabel(r"$\langle s(x)\rangle$", rotation=0, labelpad=30)
    plt.show()

    plt.plot(t, axion_bar, ".-")
    plt.xlabel(r"$t/f_a$")
    plt.ylabel(r"$\langle a(x)\rangle$", rotation=0, labelpad=30)
    plt.show()

In [ ]:
# Scaling law:
%matplotlib inline
plot_settings((10,6))

colors = ['seagreen','blue','black', 'red']
acc = ['05', '03', '015', '0075']
acc_values = [0.5, 0.3, 0.15, 0.075]
runs = [0, 1, 2, 3, 4]
for i in range(len(acc)):
    for j in range(len(runs)):
        ret = pd.read_csv(f"3DN512_{acc[i]}F_ACC/3DN512_{j}_output_files/time-series.csv")
        tau = a = np.array(ret['scalefactor'].tolist())
        xi = np.array(ret['xi'].tolist())

        plt.plot(tau, xi, "-", color=colors[i], label=f"{acc_values[i]}")

xi_plot_settings()
plt.ylim((1, 2))
# plt.legend()
plt.show()

In [ ]:
# Power spectrum snapshots:
%matplotlib inline
plot_settings()

ret = pd.read_csv("snapshot0-pk.csv")
k = np.array(ret['k'].tolist())
pk = np.array(ret['pk'].tolist())
n_modes = np.array(ret['n_modes'].tolist())

plt.loglog(k, pk, "-")
plt.loglog(k, pkphi(k, lambdaPRS=1.0))
power_spectrum_plot_settings()
plt.show()

plt.loglog(k, n_modes, ".")
plt.xlabel(r"$k$")
plt.ylabel("Number of modes sampled")
plt.show()

In [ ]:
# Plot string snapshot:
%matplotlib notebook
plot_settings(figsize=(8,8))

from mpl_toolkits.mplot3d import Axes3D

ret = pd.read_csv("snapshot2-strings.csv", header=None)
xs = ret[0].tolist()
ys = ret[1].tolist()
zs = ret[2].tolist()

ax = plt.axes(projection='3d')
ax.scatter3D(xs, ys, zs, c=zs, cmap=matplotlib.cm.viridis);
plt.show()

In [ ]:
# Power spectrum sanity check:
%matplotlib inline
plot_settings()

# power spectrum estimator:

def power_spectrum2D(f,N,L):
    y = fftn(f)
    P = np.abs(y)**2
    kfreq = fftfreq(N)*N

    kfreq2D = np.meshgrid(kfreq, kfreq)
    K = np.sqrt(kfreq2D[0]**2 + kfreq2D[1]**2)
    K = K.flatten()
    P = P.flatten()
    kbins = np.arange(0.5, N//2, 1.)
    kvals = 0.5 * (kbins[1:] + kbins[:-1])
    # Pk, a, b = binned_statistic(K, P, statistic="mean", bins=kbins)
    Pk, bin_edges, _ = binned_statistic(K, P, statistic = "sum", bins = kbins)
    count, _, _ = binned_statistic(K, P, statistic = "count", bins = kbins)
    for i in range(len(Pk)):
        avg_bin = 0.5 * (bin_edges[i] + bin_edges[i+1])
        Pk[i] /= 2*np.pi*avg_bin
    # Pk *= np.pi * (kbins[1:]**2 - kbins[:-1]**2)

    return kvals*2*np.pi/L, Pk, count

def power_spectrum3D(f,N,L):
    y = fftn(f)
    P = np.abs(y)**2
    kfreq = fftfreq(N)*N

    kfreq3D = np.meshgrid(kfreq, kfreq, kfreq)
    K = np.sqrt(kfreq3D[0]**2 + kfreq3D[1]**2 + kfreq3D[2]**2)
    K = K.flatten()
    P = P.flatten()
    kbins = np.arange(0.5, N//2, 1.)
    kvals = 0.5 * (kbins[1:] + kbins[:-1])
    # Pk, a, b = binned_statistic(K, P, statistic = "mean", bins = kbins)
    Pk, bin_edges, _ = binned_statistic(K, P, statistic = "sum", bins = kbins)
    count, _, _ = binned_statistic(K, P, statistic = "count", bins = kbins)
    for i in range(len(Pk)):
        avg_bin = 0.5 * (bin_edges[i] + bin_edges[i+1])
        Pk[i] /= 4*np.pi*avg_bin**2
    # Pk *= 4/3 * np.pi * (kbins[1:]**3 - kbins[:-1]**3)

    return kvals*(2*np.pi/L), Pk, count


dim = 2
N = 128
L = N * 1.0
if dim == 2:
    phi = convert_format2D(np.fromfile("snapshot0-phi1", dtype=np.float64), N)
    ks, pks, count = power_spectrum2D(phi, N, L)
else:
    phi = convert_format3D(np.fromfile("snapshot0-phi1", dtype=np.float64), N)
    ks, pks, count = power_spectrum3D(phi, N, L)

plt.loglog(ks, pks, "-")
plt.loglog(ks, pkphi(ks, lambdaPRS=0.0))
power_spectrum_plot_settings()
plt.show()

plt.loglog(ks, count, ".")
plt.xlabel(r"$k$")
plt.ylabel("Number of modes sampled")
plt.show()

In [ ]:
# Test gaussianity with no interactions:
%matplotlib inline
plot_settings((8, 8))

dim = 2
N = 512
L = N * 1.0

if dim == 2:
    snapshots = read_snapshots2D(N, func=lambda p1, p2: p1) # Just pick out phi1 field for now.
else:
    snapshots = read_snapshots3D(N, func=lambda p1, p2: p1)

timings = pd.read_csv("snapshot-timings.csv")
scalefactor = tau = np.array(timings['tau'].tolist())

for i in range(len(snapshots)):
    ks, pks, _ = power_spectrum2D(snapshots[i], N, L) if dim == 2 else power_spectrum3D(snapshots[i], N, L)
    plt.loglog(ks, pks, ".", label=fr"$\tau = {round(tau[i],2)}$")
    plt.loglog(ks, pkphi(ks, lambdaPRS=0.0), label=r"$P(k)=n_k/\omega_k$")
    plt.ylim((5e-6, 1e-2))
    power_spectrum_plot_settings()
    plt.legend()
    plt.show()

ks, pks, _ = power_spectrum2D(snapshots[0], N, L) if dim == 2 else power_spectrum3D(snapshots[0], N, L)
plt.loglog(ks, pks, "-", label=fr"$\tau/\tau_0 = {round(tau[0], 2)}$")
plt.loglog(ks, pkphi(ks, lambdaPRS=0.0), "--", label=r"$P\,(k)=n_k/\omega_k$")

ks, pks, _ = power_spectrum2D(snapshots[-1], N, L) if dim == 2 else power_spectrum3D(snapshots[-1], N, L)
plt.loglog(ks, pks, "-", label=fr"$\tau/\tau_0 = {round(tau[-1], 2)}$")
power_spectrum_plot_settings()
plt.legend()
plt.show()


In [ ]:
# Visualise binary image of flagged gridpoints:
%matplotlib inline
plot_settings(figsize=(13,12))

N = 128
flagged = convert_format2D(np.fromfile("snapshot3-flagged", dtype=np.int32), N)

plt.imshow(flagged, cmap=matplotlib.cm.viridis, interpolation='none', vmin=0, vmax=1)
plt.colorbar(fraction=0.046, pad=0.04)
plt.show()


In [ ]:
# Sobel convolution:
%matplotlib inline
plot_settings(figsize=(13,12))

def fft_kernel(kernel):
    sz = (flagged.shape[0] - kernel.shape[0], flagged.shape[1] - kernel.shape[1])  # total amount of padding
    kernel = np.pad(kernel, (((sz[0]+1)//2, sz[0]//2), ((sz[1]+1)//2, sz[1]//2)), 'constant')
    kernel = fftshift(kernel)
    kernel_fft = fftn(kernel)
    kernel_fft = fftshift(kernel_fft)
    return kernel_fft

N = 128
flagged = convert_format2D(np.fromfile("snapshot-flagged", dtype=np.int32), N)
flagged_fft = fftn(flagged)
flagged_fft = fftshift(flagged_fft)

sobel_x = np.array([[-1,0,1],[-2,0,2],[-1,0,1]])
sobel_y = np.array([[-1,-2,-1],[0,0,0],[1,2,1]])

sobel_x_fft = fft_kernel(sobel_x)
sobel_y_fft = fft_kernel(sobel_y)

flagged_fft_x = np.copy(flagged_fft)
flagged_fft_y = np.copy(flagged_fft)

# convolution
xs = np.arange(-N/2, N/2)
ys = np.arange(-N/2, N/2)
for i in range(N):
    for j in range(N):
        flagged_fft_x[i,j] *= sobel_x_fft[i,j]
        flagged_fft_y[i,j] *= sobel_y_fft[i,j]

flagged_fft_x = fftshift(flagged_fft_x)
flagged_fft_y = fftshift(flagged_fft_y)

flagged_ifft_x = ifftn(flagged_fft_x)
flagged_ifft_y = ifftn(flagged_fft_y)

plt.imshow(np.abs(flagged_ifft_x)**2 + np.abs(flagged_ifft_y)**2, cmap=matplotlib.cm.viridis, interpolation='none')
plt.colorbar(fraction=0.046, pad=0.04)
plt.show()

In [ ]:
# Gaussian convolution and peak finding:
%matplotlib inline
plot_settings(figsize=(13,12))

N = 128

flagged = convert_format2D(np.fromfile("snapshot-on-failure-flagged", dtype=np.int32), N)

flagged_fft = fftn(flagged)
flagged_fft = fftshift(flagged_fft)

# convolution
xs = np.arange(-N/2, N/2)
ys = np.arange(-N/2, N/2)
for i in range(N):
    for j in range(N):
        flagged_fft[i,j] *= np.exp(-0.01*(xs[i]**2 + ys[j]**2))

flagged_ifft = ifftn(flagged_fft)

def find_local_maxima(field, N):
    ret = []
    for i in range(N):
        for j in range(N):
            v = field[i,j]
            threshold = 0.001
            if v > threshold and v > field[(i+1)%N,j] and v > field[(i-1)%N,j] and v > field[i,(j+1)%N] and v > field[i,(j-1)%N]:
                ret.append((i,j))
    return ret

ret = find_local_maxima(np.abs(flagged_ifft), N)

plt.imshow(np.abs(flagged_ifft), cmap=matplotlib.cm.viridis, interpolation='none')
plt.colorbar(fraction=0.046, pad=0.04)
xs = [t[0] for t in ret]
ys = [t[1] for t in ret]
plt.scatter(ys, xs,color='r')
plt.show()
